In [ ]:
import pandas as pd
from tqdm import tqdm
from utils.evaluator import evaluate
dataset = 'Apache'
output_file = f'outputs/parser/LogBatcher_0shot_32candidate_10batchsize_2000chunksize_full/{dataset}_2k.log_structured.csv'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
# a =evaluate(output_file=output_file, groundtruth_file=groundtruth_file,dataset=dataset)
# print(a)
df1 = pd.read_csv(output_file)
logs1 = df1['EventTemplate'].tolist()
df2 = pd.read_csv(groundtruth_file)
logs2 = df2['EventTemplate'].tolist()

count = 0
length = len(logs1)
for i in tqdm(range(length)):
    if logs1[i] == logs2[i]:
        count += 1
print(count / length)
print(count)


In [ ]:
from utils.postprocess import correct_single_template

print(correct_single_template('<*> "<*> <*> 1.2.3.4:<*> status: <*> len: <*> time: <*>'))

In [ ]:
import re
import pandas as pd
dataset = 'HDFS'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
df = pd.read_csv(groundtruth_file)
# 定义更新规则的函数
def update_content(content):
    # 使用正则表达式匹配并更新内容
    pattern = r'(BLOCK\* NameSystem\.allocateBlock: [^ ]*) ([^ ]*)'
    replacement = r'\1. \2'
    return re.sub(pattern, replacement, content)

# 更新Content列
df['Content'] = df['Content'].apply(update_content)

# 保存修改后的CSV文件
df.to_csv('updated_file.csv', index=False)

In [ ]:
import pandas as pd
from tqdm import tqdm
from utils.evaluator import evaluate
dataset = 'BGL'
output_file = f'outputs/parser/LILAC_full/{dataset}_full.log_structured.csv'
# output_file ='outputs/parser/LogBatcher_0shot_32candidate_10batchsize/BGL_2k.log_structured.csv'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
# groundtruth_file = 'dataset/BGL/BGL_2k.log_structured_corrected.csv'
a =evaluate(output_file=output_file, groundtruth_file=groundtruth_file,dataset=dataset)
print(a)




In [14]:
cost  =[[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `www.dropbox.com:443 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} open through proxy {variables} HTTPS`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn.tencent.com:443 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} open through proxy {variables} HTTPS`"
    },
    {
        "role": "user",
        "content": "Log message: `183.60.49.182:443 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} open through proxy {variables} HTTPS`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS`"
    }
],
[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 0 bytes sent, 0 bytes received, lifetime 00:14`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes sent, {variables} bytes received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 653 bytes sent, 358 bytes received, lifetime 00:02`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes sent, {variables} bytes received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 803 bytes sent, 186 bytes received, lifetime 00:10`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes sent, {variables} bytes received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 0 bytes sent, 0 bytes received, lifetime 00:01`"
    }
],
[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 2449 bytes (2.39 KB) sent, 730 bytes received, lifetime <1 sec`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes ({variables}) sent, {variables} bytes received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 431 bytes sent, 10455 bytes (10.2 KB) received, lifetime <1 sec`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes sent, {variables} bytes ({variables}) received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 1216 bytes (1.18 KB) sent, 366 bytes received, lifetime <1 sec`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes ({variables}) sent, {variables} bytes received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 451 bytes sent, 18846 bytes (18.4 KB) received, lifetime <1 sec`"
    }
],
[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 653 bytes sent, 358 bytes received, lifetime 00:02`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes sent, {variables} bytes received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 1468 bytes (1.43 KB) sent, 7090 bytes (6.92 KB) received, lifetime 00:30`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes ({variables}) sent, {variables} bytes ({variables}) received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 2998 bytes (2.92 KB) sent, 163400 bytes (159 KB) received, lifetime 00:01`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes ({variables}) sent, {variables} bytes ({variables}) received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 1190 bytes (1.16 KB) sent, 1671 bytes (1.63 KB) received, lifetime 00:02`"
    }
],[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 2449 bytes (2.39 KB) sent, 730 bytes received, lifetime <1 sec`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes ({variables}) sent, {variables} bytes received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 431 bytes sent, 10455 bytes (10.2 KB) received, lifetime <1 sec`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes sent, {variables} bytes ({variables}) received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 1216 bytes (1.18 KB) sent, 366 bytes received, lifetime <1 sec`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} close, {variables} bytes ({variables}) sent, {variables} bytes received, lifetime {variables}`"
    },
    {
        "role": "user",
        "content": "Log message: `proxy.cse.cuhk.edu.hk:5070 close, 1165 bytes (1.13 KB) sent, 815 bytes received, lifetime <1 sec`"
    }
],[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `183.62.156.108:22 open through proxy socks.cse.cuhk.edu.hk:1080 SOCKS5`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} open through proxy {variables} SOCKS5`"
    },
    {
        "role": "user",
        "content": "Log message: `183.62.156.108:22 open through proxy socks.cse.cuhk.edu.hk:1080 SOCKS5`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} open through proxy {variables} SOCKS5`"
    },
    {
        "role": "user",
        "content": "Log message: `183.62.156.108:22 open through proxy socks.cse.cuhk.edu.hk:1080 SOCKS5`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} open through proxy {variables} SOCKS5`"
    },
    {
        "role": "user",
        "content": "Log message: `183.62.156.108:22 open through proxy socks.cse.cuhk.edu.hk:5070 SOCKS5`"
    }
],[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn6.tencent.com:443 error : A connection request was canceled before the completion.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : A connection request was canceled before the completion.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn6.tencent.com:443 error : A connection request was canceled before the completion.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : A connection request was canceled before the completion.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn6.tencent.com:443 error : A connection request was canceled before the completion.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : A connection request was canceled before the completion.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn6.tencent.com:443 error : A connection request was canceled before the completion.`"
    }
],[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:80 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:443 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:443 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:80 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    }
],[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:80 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:443 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:443 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `formi.baidu.com:843 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403`"
    }
],[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:80 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:443 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:443 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `223.167.104.147:80 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001`"
    }
],[
    {
        "role": "system",
        "content": "You are an expert of log parsing, and now you will help to do log parsing."
    },
    {
        "role": "user",
        "content": "I want you to act like an expert of log parsing. I will give you a log message delimited by backticks. You must identify and abstract all the dynamic variables in logs with {placeholder} and output a static log template. Print the input log's template delimited by backticks."
    },
    {
        "role": "assistant",
        "content": "Sure, I can help you with log parsing."
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:80 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:443 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `tcpconn4.tencent.com:443 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "assistant",
        "content": "Log template: `{variables} error : Could not connect through proxy {variables} - Proxy closed the connection unexpectedly.`"
    },
    {
        "role": "user",
        "content": "Log message: `qa.sockets.stackexchange.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061`"
    }
]]

LogBatcher_0shot_32candidate_10batchsize_2000chunksize_full_time

In [15]:
import json

import tiktoken
def count_message_tokens(messages, model_name):
    # 根据模型名称加载合适的编码器
    if model_name == "gpt-4":
        encoder = tiktoken.encoding_for_model("gpt-4")
    elif model_name == "gpt-3.5-turbo":
        encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
    else:
        raise ValueError("未知的模型名称")

    # 初始化token计数
    token_count = 0

    # 计算每个消息的token数
    for message in messages:
        role_tokens = encoder.encode(message['role'])
        content_tokens = encoder.encode(message['content'])
        token_count += len(role_tokens) + \
            len(content_tokens) + 4  # 加上特殊的消息分隔符的token数

    return token_count

dataset = 'BGL'

counts_token = 0
counts_message = 0

# 存储解析后的日志列表
message_list = []
# load every message
with open('outputs/cost/cost_divlog_for_Proxifier.json', 'r') as file:
    for line in file:
        if line.strip() == '[':
            list_str = ''
            start_load = True
        if line.strip() == ']':
            list_str += line
            message = json.loads(list_str)
            message_list.append(message)
            start_load = False
        if start_load:
            list_str += line
print(len(message_list))

# remove the same log messages
# def make_hashable(log_list):
#     return tuple(tuple(sorted(d.items())) for d in log_list)
# message_list = list(set(make_hashable(log_list) for log_list in message_list))
# message_list = [list(map(dict, log_list)) for log_list in message_list]
# print(len(message_list))

for message in cost:
    counts_token += count_message_tokens(message, 'gpt-3.5-turbo')
    counts_message += 1
print(counts_token, counts_token/counts_message)


1
3802 345.6363636363636


In [6]:
if -1:
    print(1)

1


In [ ]:
import string
import re

def clean(s):
    """ Preprocess log message
    Parameters
    ----------
    s: str, raw log message
    Returns
    -------
    str, preprocessed log message without number tokens and special characters
    """
    s = re.sub(':|\(|\)|=|,|"|\{|\}|@|$|\[|\]|\||;|\.', ' ', s)
    s = " ".join([word.lower() if word.isupper() else word for word in s.strip().split()])
    s = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', s))
    s = " ".join([word for word in s.split() if not bool(re.search(r'\d', word))])
    trantab = str.maketrans(dict.fromkeys(list(string.punctuation)))
    s = s.translate(trantab)
    s = " ".join([word.lower().strip() for word in s.strip().split()])
    return s

print(clean('authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root'))
print(clean('10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2580249'))

In [ ]:
import re
p_tokens = ['HTTP/1.<*>']
new_p_tokens = []
for p_token in p_tokens:
    if re.match(r'^(\/[^\/]+)+$', p_token):
        p_token = '<*>'
    new_p_tokens.append(p_token)
print(new_p_tokens)

In [ ]:
import re
line = 'iar 00106448 dear 0246dacc'

Rexs =  [
            "((?<=[^A-Za-z0-9])|^)(([0-9a-f]{2,}:){3,}([0-9a-f]{2,}))((?=[^A-Za-z0-9])|$)",
            "((?<=[^A-Za-z0-9])|^)(\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3})((?=[^A-Za-z0-9])|$)",
            "((?<=[^A-Za-z0-9])|^)([0-9a-f]{6,} ?){3,}((?=[^A-Za-z0-9])|$)",
            "((?<=[^A-Za-z0-9])|^)([0-9A-F]{4} ?){4,}((?=[^A-Za-z0-9])|$)",
            "((?<=[^A-Za-z0-9])|^)(0x[a-f0-9A-F]+)((?=[^A-Za-z0-9])|$)",
            "((?<=[^A-Za-z0-9])|^)([\\-\\+]?\\d+)((?=[^A-Za-z0-9])|$)",
            "(?<=executed cmd )(\".+?\")",
            "(\\w+):\\/\\/([^/:]+)(:\\d*)?([^# ]*)",
            "\\/[<\\*>|\\w]+\\/([<\\*>|\\w]+\\/?)+",
            "(<\\*>)(\\s*<\\*>){4,}",
            "(<\\*>)(##*<\\*>){2,}",
            "(\\w{0,1}<\\*>-){2,}(\\w|<\\*>)*",
            "<\\*>(\\.\\w+|\\(\\)|\\/)",
            "(false|true|root|ROOT)"
        ]
for rex in Rexs:
    line = re.sub(rex, "", line)
print(line)

In [9]:
import pandas as pd
dataset = 'proxifier'
df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected__.csv')

# def modify_content(row):
#     content = row['Content']
#     if 'lifetime ' in content:
#         # if 'KB) received' in content and not 'KB) sent' in content and not 'MB' in content and not '<' in content:
#         #     print('chage')
#         #     return '<*> close, <*> bytes sent, <*> bytes (<*> KB) received, lifetime <*>'
        
#         if not 'KB' in content and not '<' in content:
#             return '<*> close, <*> bytes sent, <*> bytes received, lifetime <*>'
#         if 'KB) sent' in content and not 'KB) received' in content and '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes (<*> KB) sent, <*> bytes received, lifetime <1 sec'
#         if not 'KB) sent' in content and 'KB) received' in content and '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes sent, <*> bytes (<*> KB) received, lifetime <1 sec'
#         if 'KB) sent' in content and not 'KB) received' in content and not '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes (<*> KB) sent, <*> bytes received, lifetime <*>'
#         if not 'KB) sent' in content and 'KB) received' in content and not '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes sent, <*> bytes (<*> KB) received, lifetime <*>'
#         if 'KB) sent' in content and 'KB) received' in content and not '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> KB) received, lifetime <*>'
#         if 'KB) sent' in content and 'KB) received' in content and '<' in content and not 'MB' in content:
#             return '<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> KB) received, lifetime <1 sec'
#         if 'KB) sent' in content and 'MB) received' in content and not '<' in content:
#             return '<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> MB) received, lifetime <*>'
#         else:
#             return '<*> close, <*> bytes sent, <*> bytes received, lifetime <1 sec'
#     else:
#         return row['EventTemplate']
def modify_content(template):
    return template.replace('<1 sec', '<*>')
df['EventTemplate'] = df['EventTemplate'].apply(modify_content)
df.to_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv', index=False)

In [13]:
import re
import tiktoken
import json



def count_prompt_tokens(prompt, model_name):
    """
    Count the number of tokens in the prompt
    Models supported: gpt-4, gpt-3.5-turbo
    """
    if model_name == "gpt-4":
        encoder = tiktoken.encoding_for_model("gpt-4")
    elif model_name == "gpt-3.5-turbo":
        encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
    else:
        raise ValueError("未知的模型名称")

    # 计算编码后的token数
    prompt_tokens = encoder.encode(prompt)
    return len(prompt_tokens)

with open('outputs/cost/cost_divlog_for_Proxifier.json', 'r') as file:
    prompts = json.load(file)

count = 0
for prompt in prompts:
    count += count_prompt_tokens(prompt, 'gpt-3.5-turbo') 
print(count)    
print(count/len(prompts))
print(len(prompts))

493483
246.7415
2000


In [10]:
from utils.matching import extract_variables
from utils.cluster import tokenize
import re
log1 = 'proxy.cse.cuhk.edu.hk:5070 close, 1143 bytes (1.11 KB) sent, 365 bytes received, lifetime 00:01'
log2 = 'authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 user=<*>'
template = 'authentication failure; logname= uid=<*> euid=<*> tty=NODEVssh ruser= rhost=<*>'


groups = extract_variables(log2, template)
print(groups)
if groups:
    for group in groups:
        if len(tokenize(group)) >= 2:
            print('no')
        else:
            print(tokenize(group))

('0', '0', '218.188.2.4 user=<*>')
['0']
['0']
['user']
